In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-15 21:31:38--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-11-15 21:31:38 (10.6 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLMusic").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.types import *  
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# df = spark.read.option('header', 'true').csv(SparkFiles.get(".csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
# df.show(10)


df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")

 

df = df.withColumn("customer_id",df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",df["review_date"].cast(DateType()))

df.printSchema()
# df.show(10)


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Count the number of the reviews 
df.count()

4751577

In [ ]:
# Drop duplicates and incomplete rows 
drop_df = df.dropna()
print(drop_df.count())

clean_df = drop_df.dropDuplicates()
print(clean_df.count())

4750852
4750852


In [ ]:
# Check the schema
clean_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#Transform DataFrame 
# Create a new DataFrame for review info
review_id_table= clean_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)



+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006RKX3L8DUL|   44975980|B0000508NI|     290527326| 2001-09-09|
|R100K51FJOG6EH|   30942723|B0007WF1XC|      87627244| 2014-05-15|
|R100RC88EIM93Q|   25454533|B00001P4TH|     390894852| 2004-08-04|
|R101EQALRNH5JJ|   36291899|B0028EQMW6|      13792258| 2009-08-27|
|R101O0WK9J0D53|   34896001|B00723NWJE|     214366859| 2012-06-30|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
products_df = clean_df.select(["product_id", "product_title"])
products = products_df.dropDuplicates(["product_id"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0005072271|Action Bible Song...|
|1070313335|CTI All Stars - L...|
|5552936752|Scherrer/Fritz: S...|
|5557497749|Something Out of ...|
|6301936590|       Sunny Side Up|
+----------+--------------------+
only showing top 5 rows



In [ ]:
customer_count = clean_df.select(["customer_id"]).count()
customer_count

4750852

In [ ]:
# Create a new DataFrame for customers
customer_counts = clean_df.groupBy('customer_id').count()
customers = customer_counts.withColumnRenamed('count', 'customer_count')
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   20620530|            38|
|   32978648|             1|
|   40637095|            11|
|   43560898|            23|
|   41687227|            55|
|   32347287|             9|
|   52675616|             3|
|   50749945|             1|
|   28272716|             9|
|   33757567|             6|
|   46020934|             8|
|   12940528|             2|
|   23747970|             2|
|   17233983|             1|
|   11050100|             1|
|   17271323|             7|
|   38145086|             2|
|   34501870|           239|
|   38679000|             8|
|   30832525|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
vine = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006RKX3L8DUL|          5|            2|          3|   N|
|R100K51FJOG6EH|          5|            0|          0|   N|
|R100RC88EIM93Q|          5|            1|          1|   N|
|R101EQALRNH5JJ|          2|            2|         10|   N|
|R101O0WK9J0D53|          3|            0|          0|   N|
|R102GEW0PCAJLT|          4|           36|         46|   N|
|R102ISK6MRCKF1|          5|            0|          0|   N|
|R102JWERA503N7|          5|            1|          2|   N|
|R102SARIV0G55W|          5|            1|          2|   N|
|R102SXNAQQ44XO|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
mode="append"
jdbc_url = "jdbc:postgresql://:5432/mypostgres"
config = {"user": " ",
          "password": " ",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame table to RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)